**Programador:** Marco Gutierrez. Si se tiene alguna consulta, mandarla al siguiente [email](mailto:mgutierrezc@up.edu.pe)

# Obteniendo la información para el Scrapping

Importamos las librerias para el scrapping

In [1]:
import pandas as pd
import string
import WazeRouteCalculator
import logging
import numpy as np
import os
import re

## Importando las provincias/distritos a analizar

Primero, leeremos un archivo que contiene los elementos de los ubigeos del INEI. Se puede descargar entrando [aquí](http://webinei.inei.gob.pe:8080/sisconcode/proyecto/index.htm?proyectoTitulo=UBIGEO&proyectoId=3) y dando click a la opción **Excel** en la pestaña *Busqueda por Ubicación Geográfica*

In [2]:
ubigeos = pd.read_excel('ubigeo_inei.xls',skiprows=1)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,01 Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,02 Asunción
4,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,03 Balsas


Necesitamos las provincias y distritos de esta base, así que limpiaremos los valores de estas variables

In [3]:
ubigeos["DISTRITO"] = ubigeos["DISTRITO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["PROVINCIA"] = ubigeos["PROVINCIA"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["DEPARTAMENTO"] = ubigeos["DEPARTAMENTO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,Amazonas,NaN,NaN,,
1,NaN,Amazonas,NaN,NaN,Chachapoyas,
2,NaN,Amazonas,NaN,NaN,Chachapoyas,Chachapoyas
3,NaN,Amazonas,NaN,NaN,Chachapoyas,Asunción
4,NaN,Amazonas,NaN,NaN,Chachapoyas,Balsas


Limpiando las listas

Necesitaremos estas dos columnas como listas, pues después las usaremos para crear un DataFrame donde guardaremos los resultados del scrapping

In [4]:
# Creando las listas
provincias = list(ubigeos['PROVINCIA'])
distritos = list(ubigeos['DISTRITO'] + ", " + ubigeos['PROVINCIA'] + ", " + ubigeos['DEPARTAMENTO'])
for_removal_dist = list(", " + ubigeos['PROVINCIA'] + ", " + ubigeos['DEPARTAMENTO'])

In [5]:
# Eliminando duplicados
provincias = set(provincias)
distritos = set(distritos)
for_removal_dist = set(for_removal_dist)

# Eliminando algunos elementos inutiles ('', nan)
provincias = list(filter(lambda x: str(x) != 'nan', provincias))
distritos = list(filter(lambda x: str(x) != 'nan', distritos))

provincias.remove('')
provincias.remove('PROVINCIA')
distritos = [distr for distr in distritos if distr not in for_removal_dist]
distritos.remove('DISTRITO, PROVINCIA, DEPARTAMENTO')

Eliminando las tildes

In [6]:
# Definimos una función para eliminar las tildes
def normalize(s):
    replacements = (
        ("á", "a"),
        ("à", "a"),
        ("é", "e"),
        ("è", "e"),
        ("í", "i"),
        ("ì", "i"),
        ("ó", "o"),
        ("ò", "o"),
        ("ú", "u"),
        ("ù", "u")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [7]:
# La aplicamos a cada elemento de nuestras provincias/distritos
nro_provs = len(provincias)
for i in range(nro_provs):
    provincias[i] = normalize(provincias[i])
    # De paso, les quitaremos espacios
    provincias[i] = provincias[i].strip()

nro_distrs = len(distritos)
for i in range(nro_distrs):
    distritos[i] = normalize(distritos[i])
    distritos[i] = distritos[i].split(",")
    
    # Eliminando los espacios dentro de los elementos de un distrito único (distr, prov, dept)
    item_distr = len(distritos[i])
    empty_distr = []
    for j in range(item_distr):
        distritos[i][j] = distritos[i][j].strip()
        empty_distr.append(distritos[i][j])
        
    # Joining our separated districts
    distritos[i] = ','.join(empty_distr)

# Scrapping de Distancias y Tiempos de viaje

Para realizar este scrapping, se tomará como referencia este [paquete](https://github.com/kovacsbalu/WazeRouteCalculator)

## Matrices base a nivel de distritos

Crearemos nuestra matriz de la siguiente manera

In [8]:
first_col = ["Origin Province\Destination Province"]
dist_distances = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_times = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_distances

,Origin Province\Destination Province,"Llipa,Ocros,Ancash","Cascapara,Yungay,Ancash","Socos,Huamanga,Ayacucho","Lomas,Caraveli,Arequipa","Santa Isabel de Siguas,Arequipa,Arequipa","Rumisapa,Lamas,San Martin","Obas,Yarowilca,Huanuco","Talavera,Andahuaylas,Apurimac","Chaparra,Caraveli,Arequipa",...,"Santa Ana de Tusi,Daniel Alcides Carrion,Pasco","Huanca-Huanca,Angaraes,Huancavelica","Anchonga,Angaraes,Huancavelica","Honoria,Puerto Inca,Huanuco","Goyllarisquizga,Daniel Alcides Carrion,Pasco","La Yarada los Palos,Tacna,Tacna","Puyusca,Parinacochas,Ayacucho","Maria Parado de Bellido,Cangallo,Ayacucho","Huayan,Huarmey,Ancash","San Jeronimo,Cusco,Cusco"
0,"Llipa,Ocros,Ancash",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Cascapara,Yungay,Ancash",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Socos,Huamanga,Ayacucho",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Lomas,Caraveli,Arequipa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Santa Isabel de Siguas,Arequipa,Arequipa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,"La Yarada los Palos,Tacna,Tacna",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1870,"Puyusca,Parinacochas,Ayacucho",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871,"Maria Parado de Bellido,Cangallo,Ayacucho",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1872,"Huayan,Huarmey,Ancash",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creando listas con los origenes/destinos para reemplazar en el dataframe

In [9]:
# Como queremos iterar sobre los nombres de las provincias
origen_destino_dist=[]

for dist_row in distritos:
    origen_destino_dist.append(dist_row+","+"Peru")

origen_destino_dist

['Llipa,Ocros,Ancash,Peru',
 'Cascapara,Yungay,Ancash,Peru',
 'Socos,Huamanga,Ayacucho,Peru',
 'Lomas,Caraveli,Arequipa,Peru',
 'Santa Isabel de Siguas,Arequipa,Arequipa,Peru',
 'Rumisapa,Lamas,San Martin,Peru',
 'Obas,Yarowilca,Huanuco,Peru',
 'Talavera,Andahuaylas,Apurimac,Peru',
 'Chaparra,Caraveli,Arequipa,Peru',
 'Quequeña,Arequipa,Arequipa,Peru',
 'Pitumarca,Canchis,Cusco,Peru',
 'Emilio San Martin,Requena,Loreto,Peru',
 'Huacullani,Chucuito,Puno,Peru',
 'Juan Guerra,San Martin,San Martin,Peru',
 'Pacobamba,Andahuaylas,Apurimac,Peru',
 'Pichanaqui,Chanchamayo,Junin,Peru',
 'Nuevo Progreso,Tocache,San Martin,Peru',
 'Viru,Viru,La Libertad,Peru',
 'Toro,La Union,Arequipa,Peru',
 'Camporredondo,Luya,Amazonas,Peru',
 'Pacaraos,Huaral,Lima,Peru',
 'Tayabamba,Pataz,La Libertad,Peru',
 'El Cenepa,Condorcanqui,Amazonas,Peru',
 'Caceres del Peru,Santa,Ancash,Peru',
 'Chancay,San Marcos,Cajamarca,Peru',
 'Alto Pichigua,Espinar,Cusco,Peru',
 'Papayal,Zarumilla,Tumbes,Peru',
 'Soloco,Chachap

## ¿Cómo usar el paquete `WazeRouteCalculator`?

Ahora que ya tenemos todos los datos listos, nos falta activar la clase `WazeRouteCalculator` para que podamos extraer tiempos y distancias de waze. Ello se puede hacer así:

Obtenemos el logger de dicho paquete

In [10]:
logger = logging.getLogger('WazeRouteCalculator.WazeRouteCalculator')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
logger.addHandler(handler)

## Usando `WazeRouteCalculator` para crear una función que calcule distancias/tiempos

In [11]:
def matrix_blocks(first_row, first_column, final_row_plus_one, final_col, df_distances, df_times, origin_dest):
    """
    Función para completar la matriz por bloques que comiencen desde "first_row" y "first_column" hasta "iterations_row" 
    y "iterations_column". Se creó porque ejecutar el sgte codigo para todas las celdas toma mucho tiempo en ejecutarse.
    
    Para hacer más eficiente la tarea, se puede ejecutar esto por partes hasta completarlo. Además, permite convertir la tarea
    en una grupal. Un equipo puede hacer que cada uno de sus miembros complete ciertos bloques (e.g. linea 1 hasta 12, otro 
    linea 12 hasta 24, y así sucesivamente)
    """
    # Codigo base para cada fila (comenzando con la 0 y recordando que range tomará como ultimo valor el limite superior - 1)
    for row in range(first_row, final_row_plus_one):
        print(row)
        for col in range(first_column, final_col):
            # Definiendo un índice para nuestra lista con los origenes/destinos en función a la columna del dataframe
            # en la que queremos el resultado
            index_destination = col-1

            # Si las filas y columnas son consecutivas, las distancias y tiempo son de un sitio al mismo sitio, asi que 
            # debemos controlar ese factir
            if col-1==row:
                df_distances.iloc[row,col] = 0
                df_times.iloc[row,col] = 0

            else:
                # Obteniendo las distancias/tiempos correspondientes (el default es manejando un auto privado): 
                route = WazeRouteCalculator.WazeRouteCalculator(origin_dest[row], origin_dest[index_destination])
                # Como estamos en cuarentena (11-04-2020), no debemos colocar tiempo real, pues al no haber las restricciones
                # normales de viaje, habrá un sesgo negativo en el tiempo
                try:
                    route_results = route.calc_route_info(real_time=False)
                except:
                    route_results = [0, 0]
                # Usaremos diccionarios en lugar de variables simples pues permiten usar la func '.get'
                # La distancia está en km y la duración en minutos
                distance_duration = {'distance': route_results[1], 'duration': route_results[0]}
                
                # No siempre es posible viajar de un lugar a otro por restricciones geográficas, así que debemos definir qué ocurre
                # en casos así
                if distance_duration.get('distance'):
                    df_distances.iloc[row,col] = round(distance_duration['distance'], 2)
                else:
                    df_distances.iloc[row,col] = 0

                if distance_duration.get('duration'):
                    df_times.iloc[row,col] = round(distance_duration['duration'], 2)
                else:
                    df_times.iloc[row,col] = 0            

                # Para debuggear:
                # print('Origen: '+origen_destino[row]+' | Destino: '+origen_destino[index_destination]) 
                # print(prov_distances.iloc[row,col])

### Aplicando la función a las Matrices de Distritos

In [12]:
origen_destino_dist[0]
# origen_destino_dist[1]

'Llipa,Ocros,Ancash,Peru'

In [ ]:
# La primera fila puede tomar valores comenzando desde 0 
primera_fila = 0
primera_col = 1
# Si quieres una determinada fila final, por ej, fila 2 (comenzando en 0), súmale uno a su valor 
# y registralo aquí (ult_fila_mas_uno = 3)
ult_fila_mas_uno = 10
ult_col = len(distritos)+1
# ult_col = 4

matrix_blocks(primera_fila, primera_col, ult_fila_mas_uno, ult_col, dist_distances, dist_times, origen_destino_dist)

From: Llipa,Ocros,Ancash,Peru - to: Cascapara,Yungay,Ancash,Peru


0


Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-9.139538764953613, -77.74421691894531)
From: Llipa,Ocros,Ancash,Peru - to: Socos,Huamanga,Ayacucho,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-13.16025161743164, -74.22490692138672)
From: Llipa,Ocros,Ancash,Peru - to: Lomas,Caraveli,Arequipa,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-16.40382957458496, -71.52893829345703)
From: Llipa,Ocros,Ancash,Peru - to: Santa Isabel de Siguas,Arequipa,Arequipa,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-6.484586238861084, -79.9627914428711)
From: Llipa,Ocros,Ancash,Peru - to: Rumisapa,Lamas,San Martin,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-5.688209533691406, -78.79151916503906)
From: Llipa,Ocros,Ancash,Peru - to: Obas,Yarowilca,Huanuco,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-9.878246307373047, -76.20684051513672)
From: Llip

End coords: (-11.957736015319824, -77.07015228271484)
From: Llipa,Ocros,Ancash,Peru - to: Chugur,Hualgayoc,Cajamarca,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-7.156628131866455, -78.516357421875)
From: Llipa,Ocros,Ancash,Peru - to: Lamud,Luya,Amazonas,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-6.144145965576172, -77.94881439208984)
From: Llipa,Ocros,Ancash,Peru - to: San Miguel,La Mar,Ayacucho,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-13.012399673461914, -73.9810562133789)
From: Llipa,Ocros,Ancash,Peru - to: Magdalena de Cao,Ascope,La Libertad,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-7.713854789733887, -79.10908508300781)
From: Llipa,Ocros,Ancash,Peru - to: Pachamarca,Churcampa,Huancavelica,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-5.196068286895752, -80.63069152832031)
From: Llipa,Ocros,Ancash,Peru - to: Tinyahuarco,Pasco,Pasco

From: Llipa,Ocros,Ancash,Peru - to: Eduardo Villanueva,San Marcos,Cajamarca,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-6.778202533721924, -79.83708953857422)
From: Llipa,Ocros,Ancash,Peru - to: Pampas de Hospital,Tumbes,Tumbes,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-3.5732338428497314, -80.419921875)
From: Llipa,Ocros,Ancash,Peru - to: Yanaca,Aymaraes,Apurimac,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-12.596400260925293, -69.1945571899414)
From: Llipa,Ocros,Ancash,Peru - to: Sachaca,Arequipa,Arequipa,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-16.373313903808594, -71.5468521118164)
From: Llipa,Ocros,Ancash,Peru - to: Aplao,Castilla,Arequipa,Peru
Start coords: (-10.403828620910645, -77.3971176147461)
End coords: (-16.40302085876465, -71.51643371582031)
From: Llipa,Ocros,Ancash,Peru - to: Yauli,Jauja,Junin,Peru
Start coords: (-10.403828620910645, -77.397117614746

...

8


Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-10.403828620910645, -77.3971176147461)
From: Chaparra,Caraveli,Arequipa,Peru - to: Cascapara,Yungay,Ancash,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-9.139538764953613, -77.74421691894531)
Time 1345.10 minutes, distance 1468.76 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Socos,Huamanga,Ayacucho,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-13.16025161743164, -74.22490692138672)
Time 1045.55 minutes, distance 1118.06 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Lomas,Caraveli,Arequipa,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-16.40382957458496, -71.52893829345703)
Time 0.30 minutes, distance 0.06 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Santa Isabel de Siguas,Arequipa,Arequipa,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-6.484586238861084, -79.9627914428711)
Time 1628.93 minutes, distanc

Time 1310.85 minutes, distance 1379.63 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Ongoy,Chincheros,Apurimac,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-13.39067268371582, -72.0478744506836)
Time 632.43 minutes, distance 640.91 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Soritor,Moyobamba,San Martin,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-5.688209533691406, -78.79151916503906)
Time 1874.93 minutes, distance 2085.66 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: San Isidro de Maino,Chachapoyas,Amazonas,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-6.337203025817871, -77.88031005859375)
Time 2032.12 minutes, distance 2261.69 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Ñahuimpuquio,Tayacaja,Huancavelica,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-5.196068286895752, -80.63069152832031)
Time 1742.57 minutes, distance 1996.89 km.
From: Chaparra,Caraveli,Arequ

Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-15.837499618530273, -70.02278900146484)
Time 304.28 minutes, distance 298.54 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Jose Manuel Quiroz,San Marcos,Cajamarca,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-7.152938365936279, -78.5204849243164)
Time 1709.92 minutes, distance 1877.31 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: La Morada,Marañon,Huanuco,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-12.138527870178223, -76.82549285888672)
Time 892.20 minutes, distance 993.99 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: La Esperanza,Trujillo,La Libertad,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-8.054444313049316, -79.06783294677734)
Time 1384.68 minutes, distance 1587.08 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Chincha Alta,Chincha,Ica,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-13.417096

End coords: (-16.333080291748047, -71.5477066040039)
Time 27.68 minutes, distance 11.00 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Contumaza,Contumaza,Cajamarca,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-7.362380504608154, -78.79637908935547)
Time 1493.52 minutes, distance 1720.87 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Madre de Dios,Manu,Madre de Dios,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-12.618550300598145, -70.39421844482422)
Time 668.08 minutes, distance 732.37 km.
From: Chaparra,Caraveli,Arequipa,Peru - to: Tingo de Ponasa,Picota,San Martin,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-6.514010429382324, -76.74082946777344)
From: Chaparra,Caraveli,Arequipa,Peru - to: Calango,Cañete,Lima,Peru
Start coords: (-16.40382957458496, -71.52893829345703)
End coords: (-12.047479629516602, -77.04093933105469)
Time 904.05 minutes, distance 1012.35 km.
From: Chaparra,Caraveli,Arequipa,Pe

In [ ]:
ultima_fila = ult_fila_mas_uno - 1
dist_distances.to_csv('district_data\\dist_distances_ini-{}_fin-{}.csv'.format(primera_fila, ultima_fila), index=False)
dist_times.to_csv('district_data\\dist_times_ini-{}_fin-{}.csv'.format(primera_fila, ultima_fila), index=False)